In [16]:
#일반적인 인공 신경망은 다양한 형태의 입력에 대한 확장성이 떨어진다. -> 컨볼루션 :필터를 적용. 이 필터를 학습
#컨볼루션 레이어 : 이미지의 특징을 추출
#풀링 레이어 : 중요한 특징을 골라낸다. 일종의 컨볼루션
#피처 맵의 크기가 크면 학습이 어렵고 오버피팅의 위험이 증가한다.

# CNN으로 패션 아이템 구분하기
Convolutional Neural Network (CNN) 을 이용하여 패션아이템 구분 성능을 높여보겠습니다.

In [17]:
#accuracy 전체 중에 true인 분류의 비율
#recall 암환자를 한명도 놓치고 싶지 않다. 웬만하면 암이라 분류 (실제 positive가 분모:FN+TP)
#precision 나는 정확도 높은 판단을 하고 싶다. 보수적으로 판단. 확실한 경우에만 암이라 분류 (내가 positive라 예측한 것이 분모:FP+TP)
#ROC 커브 이진분류기에서 많이 사용. feature 선택에 유용 [y축:TP/(TP+FN)실제 암인데 맞춤recall], [x축:FP/(TN+FP)정상인데 암이라함]
#AUC(Area Under the Curve) : ROC커브의 면적. 클수록 decision boundary에 덜 민감
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [18]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [19]:
EPOCHS     = 40
BATCH_SIZE = 64

## 데이터셋 불러오기

In [20]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

9<00:41, 434519.71it/s]
100%|█████████▉| 26361856/26421880 [01:18<00:00, 620248.29it/s]Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw


0it [00:00, ?it/s]Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz


  0%|          | 0/29515 [00:00<?, ?it/s]

32768it [00:01, 31239.42it/s]


0it [00:00, ?it/s]Extracting ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw


  0%|          | 0/4422102 [00:00<?, ?it/s]

  0%|          | 16384/4422102 [00:00<01:23, 52988.68it/s]

  1%|          | 49152/4422102 [00:01<01:07, 64492.27it/s]

  2%|▏         | 98304/4422102 [00:01<00:56, 76370.72it/s]

  2%|▏         | 106496/4422102 [00:01<01:27, 49435.32it/s]

  4%|▍         | 188416/4422102 [00:02<01:03, 66363.50it/s]

  5%|▌         | 229376/4422102 [00:02<00:55, 75241.28it/s]

  6%|▌         | 270336/4422102 [00:02<00:46, 89005.3

## 뉴럴넷으로 Fashion MNIST 학습하기

In [21]:
class Net(nn.Module):
    def __init__(self): #학습모듈 정의
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #nn.Conv2d():입력 x를 받는 함수를 반환. 함수로 취급해도 무방. 1:입력채널, 10:출력채널
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) #필터사이즈는 (3,5)와 같은 배열을 입력해서 직사각형으로 만들수도 있음
        self.conv2_drop = nn.Dropout2d() #함수로 대체할 수 있음
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x): #데이터가 지나갈 길
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) #두번째 입력(2)는 커널의 크기. nn.MaxPool2d 같은 일반 모듈 사용가능
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320) #2차원 형태의 x를 1차원으로 차원축소 (-1:'남는 차원 모두:전체/320', 320:'x가 가진 원소 개수')
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## 하이퍼파라미터 

`to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 합니다. 일반적으로 CPU 1개만 사용할 경우 필요는 없지만, GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 합니다. 지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없습니다.

최적화 알고리즘으로 파이토치에 내장되어 있는 `optim.SGD`를 사용하겠습니다.

In [22]:
model     = Net().to(DEVICE) #모델 인스턴스 만들기
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) #최적화 함수 만들기. 모멘텀 첫등장 설명x

## 학습하기

In [23]:
def train(model, train_loader, optimizer, epoch):
    model.train() #학습 모드
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

## 테스트하기

In [24]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났습니다. 코드를 돌려서 실제로 학습이 되는지 확인해봅시다!

In [25]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.311989
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.204634
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.098022


4423680it [00:47, 223529.26it/s]Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.955576
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.921472
[1] Test Loss: 0.6689, Accuracy: 72.67%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.849364
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.608787
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.920404
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.046688
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.096114
[2] Test Loss: 0.5725, Accuracy: 77.64%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.699395
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.815541
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.742900
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.739689
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.816691
[3] Test Loss: 0.5450, Accuracy: 79.18%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.602790
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.920416
Train Epoch: 4 [

KeyboardInterrupt: 